# PART- 2 `( Handling Missing Values )`

In [17]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats.mstats import winsorize

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

%matplotlib inline
# %matplotlib notebook

plt.rcParams["figure.figsize"] = (10,6)
# plt.rcParams['figure.dpi'] = 100

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.3f' % x)

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 150

In [18]:
df = pd.read_csv("clean_scout.csv")

In [19]:
df.shape

(15919, 46)

In [20]:
df.head(3).T

,0,1,2
url,https://www.autoscout24.com//offers/audi-a1-sp...,https://www.autoscout24.com//offers/audi-a1-1-...,https://www.autoscout24.com//offers/audi-a1-sp...
make_model,Audi A1,Audi A1,Audi A1
short_description,Sportback 1.4 TDI S-tronic Xenon Navi Klima,1.8 TFSI sport,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...
body_type,Sedans,Sedans,Sedans
price,15770,14500,14640
vat,VAT deductible,Price negotiable,VAT deductible
km,"56,013 km","80,000 km","83,450 km"
kW,NaN,NaN,NaN
Type,s,s,s
Warranty,NaN,NaN,NaN


In [21]:
df.isnull().sum()/df.shape[0]*100

url                               0.000
make_model                        0.000
short_description                 0.289
body_type                         0.377
price                             0.000
vat                              28.350
km                                0.000
kW                              100.000
Type                              0.013
Warranty                         69.514
null                              0.000
Make                              0.000
Model                             0.000
Body                              0.377
Cylinders                        35.681
Fuel                              0.000
description                       0.000
Gears                            29.600
Electricity consumption          99.139
Last Service Date                96.445
Other Fuel Types                 94.472
Availability                     96.011
Last Timing Belt Service Date    99.899
Available from                   98.291
Comfort_Convenience               5.779


In [22]:
miss_val = []

[miss_val.append(i) for i in df.columns if any(df[i].isnull())]

miss_val

['short_description',
 'body_type',
 'vat',
 'kW',
 'Type',
 'Warranty',
 'Body',
 'Cylinders',
 'Gears',
 'Electricity consumption',
 'Last Service Date',
 'Other Fuel Types',
 'Availability',
 'Last Timing Belt Service Date',
 'Available from',
 'Comfort_Convenience',
 'Entertainment_Media',
 'Extras',
 'Safety_Security',
 'age',
 'Previous_Owners',
 'hp_kW',
 'Inspection_new',
 'Body_Color',
 'Paint_Type',
 'Upholstery_type',
 'Upholstery_color',
 'Nr_of_Doors',
 'Nr_of_Seats',
 'Displacement_cc',
 'Weight_kg',
 'Drive_chain',
 'cons_comb',
 'cons_city',
 'CO2_Emission',
 'Emission_Class']

In [23]:
# function for first looking to the columns

def first_looking(col):
    print("column name    : ", col)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col].isnull().sum()/df.shape[0]*100, 2))
    print("num_of_nulls   : ", df[col].isnull().sum())
    print("num_of_uniques : ", df[col].nunique())
    print(df[col].value_counts(dropna = False))

## functions to fill the missing values

In [41]:
def fill_most(df, group_col, col_name):
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [42]:
def fill_prop(df, group_col, col_name):
    '''Fills the missing values with "ffill and bfill method" according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [43]:
def fill(df, group_col1, group_col2, col_name, method): # method can be "mode" or "median" or "ffill"
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if mode2 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
                elif mode1 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
                else:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])
                
    elif method == "median":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].median()).fillna(df[cond1][col_name].median()).fillna(df[col_name].median())
                
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

## Let's examine and fill the missing values of all the columns/features one by one

## age

In [44]:
first_looking("age")

column name    :  age
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  5
1.0    4522
3.0    3674
2.0    3273
0.0    2853
-      1597
Name: age, dtype: int64


In [45]:
df['age'].fillna('-', inplace=True)

In [46]:
df["age"].value_counts(dropna=False)

1.0    4522
3.0    3674
2.0    3273
0.0    2853
-      1597
Name: age, dtype: int64

In [47]:
df.groupby("age").km.describe()

,count,unique,top,freq
age,,,,
0.0,2853,475,10 km,594
1.0,4522,2813,10 km,80
2.0,3273,1981,"30,000 km",30
3.0,3674,2198,"70,000 km",30
-,1597,53,- km,838


In [48]:
df[df["age"]=="-"]["km"].value_counts(dropna=False)

- km         838
10 km        369
1 km         146
5 km          58
20 km         32
15 km         21
0 km          19
11 km         12
8 km          11
50 km         10
100 km         8
12 km          8
7 km           7
9 km           4
3 km           4
30 km          3
4 km           3
250 km         3
25 km          3
39,962 km      2
22,627 km      2
3,000 km       2
2 km           2
6,100 km       1
500 km         1
142 km         1
11,200 km      1
32,084 km      1
141 km         1
68,485 km      1
81,800 km      1
497 km         1
60 km          1
19,500 km      1
11,000 km      1
196 km         1
40 km          1
6 km           1
784 km         1
34,164 km      1
3,500 km       1
4,500 km       1
5,000 km       1
99 km          1
89,982 km      1
20,768 km      1
77 km          1
281 km         1
150 km         1
4,307 km       1
85,000 km      1
89,692 km      1
325 km         1
Name: km, dtype: int64

In [53]:
cond1 = (df['km'] < 10000)
cond2 = ((df['km'] >= 10000) & (df['km'] < 28000))
cond3 = ((df['km'] >= 28000) & (df['km'] < 50000))
cond4 = (df['km'] >= 50000)

TypeError: '<' not supported between instances of 'str' and 'int'

In [54]:
df.loc[cond1,'age'] = df.loc[cond1,'age'].replace('-', 0)
df.loc[cond2,'age'] = df.loc[cond2,'age'].replace('-', 1)
df.loc[cond3,'age'] = df.loc[cond3,'age'].replace('-', 2)
df.loc[cond4,'age'] = df.loc[cond4,'age'].replace('-', 3)

NameError: name 'cond1' is not defined

In [55]:
df.groupby('age').km.mean()

DataError: No numeric types to aggregate

In [56]:
df["age"].value_counts(dropna=False)

1.0    4522
3.0    3674
2.0    3273
0.0    2853
-      1597
Name: age, dtype: int64

In [57]:
df.groupby(['make_model', 'age']).km.describe()

count unique         top freq
make_model     age                              
Audi A1        0.0   483    136       10 km   67
               1.0   744    608   15,000 km   11
               2.0   432    340   17,768 km   27
               3.0   629    482   63,668 km   27
               -     326     17        - km  229
Audi A2        1.0     1      1   26,166 km    1
Audi A3        0.0   519    123       10 km  108
               1.0   777    543   18,000 km   13
               2.0   675    473   30,000 km   10
               3.0   818    568  122,584 km   19
               -     308     14       10 km  118
Opel Astra     0.0   454    166       10 km   68
               1.0   727    611   29,900 km    6
               2.0   587    416    3,898 km   27
               3.0   519    421   95,000 km    6
               -     239     18        - km  132
Opel Corsa     0.0   569     97       10 km  186
               1.0   683    562       10 km   18
               2.0   382    305   72,000 km    8
               3.0   429    334   99,950 km    6
               -     156     21        - km   48
Opel Insignia  0.0   315    138        1 km   33
               1.0   767    620    3,095 km   11
               2.0   589    462   25,494 km   19
               3.0   567    440   76,000 km    9
               -     360     19        - km  223
Renault Clio   0.0   292     43       10 km  105
               1.0   573    355       10 km   43
               2.0   438    278   31,356 km   27
               3.0   412    254   49,178 km   27
               -     124     16        - km   49
Renault Duster 0.0    16      1      101 km   16
               -      18      1        - km   18
Renault Espace 0.0   205     60       10 km   37
               1.0   250    179        - km   18
               2.0   170    125   49,606 km   27
               3.0   300    231   68,570 km   11
               -      66     13        - km   24

In [58]:
df.groupby(['make_model',"body_type", 'age']).price.describe()

count      mean       std       min  \
make_model     body_type     age                                         
Audi A1        Compact       0.0 129.000 24104.558  3151.954 15550.000   
                             1.0 267.000 18599.772  2664.203 13980.000   
                             2.0 161.000 16602.807  2085.384 10999.000   
                             3.0 234.000 14532.910  1908.909  9950.000   
                             -   248.000 23340.238  3609.296 14900.000   
               Coupe         2.0   1.000 15900.000       nan 15900.000   
                             3.0   1.000 13950.000       nan 13950.000   
               Other         0.0   8.000 23826.250  2057.439 21490.000   
                             1.0   3.000 16796.667   178.979 16590.000   
                             2.0   1.000 23490.000       nan 23490.000   
                             -     1.000 21850.000       nan 21850.000   
               Sedans        0.0 343.000 24297.907  3127.912 15990.000   
                             1.0 463.000 18406.667  2587.568 13450.000   
                             2.0 266.000 16605.395  2223.979 11600.000   
                             3.0 389.000 14466.900  1890.278  8999.000   
                             -    77.000 22781.571  3878.285 15679.000   
               Station wagon 0.0   2.000 28944.500    77.075 28890.000   
                             1.0  11.000 17410.909  2166.031 15750.000   
                             2.0   3.000 17613.333  2953.817 14750.000   
                             3.0   5.000 14358.000  1276.891 12950.000   
               Van           0.0   1.000 29000.000       nan 29000.000   
Audi A2        Off-Road      1.0   1.000 28200.000       nan 28200.000   
Audi A3        Compact       0.0  15.000 30113.867  9074.515 15500.000   
                             1.0  29.000 22030.931  4270.469 14500.000   
                             2.0  39.000 20489.769  8208.445 13790.000   
                             3.0  93.000 15303.796  2256.723 12900.000   
                             -     6.000 24915.833  1496.255 22675.000   
               Convertible   1.0   3.000 29664.333 23557.901 10893.000   
                             2.0   4.000 28562.500  5301.159 20900.000   
                             3.0   1.000 20000.000       nan 20000.000   
               Coupe         2.0   1.000 19000.000       nan 19000.000   
                             3.0   3.000 17326.667   583.124 16990.000   
               Other         1.0   4.000 23422.500  3548.055 20200.000   
                             2.0   4.000 18675.000  1351.851 17650.000   
                             3.0   5.000 15970.000  2392.070 12550.000   
                             -     3.000 25990.000  1322.876 24490.000   
               Sedans        0.0 455.000 25216.418  2354.310 20000.000   
                             1.0 709.000 22544.715  5224.410 14250.000   
                             2.0 550.000 19980.358  3721.376 10550.000   
                             3.0 647.000 16702.832  2900.228 10950.000   
                             -   237.000 24178.110  2220.401 18820.000   
               Station wagon 0.0  49.000 25586.367  5871.313 17990.000   
                             1.0  32.000 21252.594  3201.415  5555.000   
                             2.0  72.000 19010.875  1522.622 14590.000   
                             3.0  67.000 15739.507  1883.567 11790.000   
                             -    62.000 23803.919  1592.986 18980.000   
Opel Astra     Compact       0.0  51.000 23056.118  7391.480 16500.000   
                             1.0  39.000 14717.538  1266.385 11999.000   
                             2.0  55.000 13091.491  1884.243  9999.000   
                             3.0  30.000 12023.733  2767.419  8450.000   
                             -    10.000 16636.600  1098.445 15816.000   
               Coupe         0.0   1.000 29990.000       nan 29990.000   
                             3.0   1.00

In [59]:
df['age'].replace('-',0, inplace=True)

In [60]:
df.groupby('age').km.mean()

DataError: No numeric types to aggregate

In [61]:
df["age"].value_counts(dropna=False)

1.000    4522
0.000    4450
3.000    3674
2.000    3273
Name: age, dtype: int64

## km

In [62]:
first_looking("km")

column name    :  km
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  6690
10 km        1045
- km         1024
1 km          367
5 km          170
50 km         148
             ... 
1,585 km        1
48,800 km       1
16,452 km       1
98,570 km       1
6,805 km        1
Name: km, Length: 6690, dtype: int64


In [63]:
df.groupby("age").km.mean()

DataError: No numeric types to aggregate

In [64]:
df.groupby("age").km.transform("mean")

DataError: No numeric types to aggregate

In [65]:
df["km"].fillna(df.groupby("age").km.transform("mean"), inplace=True)

DataError: No numeric types to aggregate

In [66]:
df.km.value_counts(dropna=False)

10 km        1045
- km         1024
1 km          367
5 km          170
50 km         148
             ... 
1,585 km        1
48,800 km       1
16,452 km       1
98,570 km       1
6,805 km        1
Name: km, Length: 6690, dtype: int64

## body type

In [67]:
first_looking("body_type")

column name    :  body_type
--------------------------------
per_of_nulls   :  % 0.38
num_of_nulls   :  60
num_of_uniques :  9
Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64


In [68]:
df.body_type.replace("Other", np.nan, inplace=True) 

In [69]:
df['body_type'].value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
NaN               350
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [70]:
df["body_type"].mode()

0    Sedans
dtype: object

In [71]:
df["body_type"].mode()[0]

'Sedans'

In [ ]:
#Step-1
#df["body_type"].fillna(df["body_type"].mode()[0])

#Step-2
#df.loc[cond, "body_type"].fillna(df[cond]["body_type"].mode()[0])

In [ ]:
for group in list(df["make_model"].unique()):
    cond = df["make_model"]==group
    mode = list(df[cond]["body_type"].mode())
    if mode != []:
        df.loc[cond, "body_type"] = df.loc[cond, "body_type"].fillna(df[cond]["body_type"].mode()[0])
    else:
        df.loc[cond, "body_type"] = df.loc[cond, "body_type"].fillna(df["body_type"].mode()[0])

In [ ]:
df['body_type'].value_counts(dropna=False)

In [ ]:
def fill_most(df, group_col, col_name):
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [ ]:
fill_most(df, "make_model", "body_type")

## Previous_Owners

In [ ]:
first_looking("Previous_Owners")

In [ ]:
df["Previous_Owners"].fillna("-", inplace = True)

In [ ]:
df["Previous_Owners"].value_counts(dropna=False)

In [ ]:
df.groupby(['make_model', 'age', 'Previous_Owners']).km.describe()

In [ ]:
df[(df["make_model"]=="Renault Duster") & (df["Previous_Owners"] == "-")]["km"]

In [ ]:
cond = (df["make_model"]=="Renault Duster") & (df["Previous_Owners"] == "-")
df.loc[cond, "Previous_Owners"] = df.loc[cond, "Previous_Owners"].replace("-", 0.0)

In [ ]:
df["Previous_Owners"].value_counts(dropna=False)

In [ ]:
df["Previous_Owners"].replace("-", np.nan, inplace=True)

In [ ]:
#Step-1
#df.["Previous_Owners"].fillna(method="ffill")

#Step-2
#df.loc[df["age"]==0, "Previous_Owners"].fillna(method="ffill")

In [ ]:
#Step-3
for group in list(df["age"].unique()):
    cond = df["age"]==group
    df.loc[cond, "Previous_Owners"] = df.loc[cond, "Previous_Owners"].fillna(method="ffill").fillna(method="bfill")
df["Previous_Owners"] = df["Previous_Owners"].fillna(method="ffill").fillna(method="bfill")

In [ ]:
df["Previous_Owners"].value_counts(dropna=False)

In [ ]:
def fill_prop(df, group_col, col_name):
    '''Fills the missing values with "ffill and bfill method" according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [ ]:
fill_prop(df, "age", "Previous_Owners")

## Warranty

In [ ]:
first_looking("Warranty")

In [ ]:
df["Warranty"].fillna("-", inplace = True)

In [ ]:
df.groupby(['make_model', 'age', 'Warranty']).price.describe()

##### There are too many nan values and when we analyzed these nan values according to the km, age and make_model columns, we decided that this column does not have healthy data.

In [ ]:
df.drop("Warranty", axis=1, inplace=True)

## vat

In [ ]:
first_looking("vat")

There is no relation between vat and other columns. So we can use ffill

In [ ]:
df.vat.fillna(method="ffill", inplace = True)

In [ ]:
df.vat.value_counts(dropna=False)

## Body_Color

In [ ]:
first_looking("Body_Color")

In [ ]:
df["Body_Color"].fillna("-", inplace = True)

In [ ]:
df["Body_Color"].value_counts(dropna=False)

In [ ]:
df.groupby(["make_model", "body_type", 'Body_Color']).price.describe()

In [ ]:
df.drop("Body_Color", axis=1, inplace=True)

### Paint Type

In [ ]:
first_looking("Paint_Type")

In [ ]:
df["Paint_Type"].fillna("-", inplace = True)

In [ ]:
df["Paint_Type"].value_counts(dropna=False)

In [ ]:
df.groupby(["make_model", "body_type", "age", 'Paint_Type']).price.describe()

In [ ]:
df["Paint_Type"].replace("-", np.nan, inplace = True)

In [ ]:
fill(df, "make_model", "body_type", "Paint_Type", "ffill")

### Type

In [ ]:
first_looking("Type")

In [ ]:
df.Type.fillna("-", inplace=True)

In [ ]:
df['Type'].value_counts(dropna=False)

In [ ]:
df.groupby(["Type", "make_model", "age"]).km.describe()

In [ ]:
cond1 = (df['make_model'] == "Audi A3") & (df["age"] == 0)
cond2 = (df['make_model'] == "Audi A3") & (df["age"] == 3)

In [ ]:
df.loc[cond1,'Type'] = df.loc[cond1,'Type'].replace('-','New')
df.loc[cond2,'Type'] = df.loc[cond2,'Type'].replace('-','Used')

In [ ]:
df['Type'].value_counts(dropna=False)

## Inspection new

In [ ]:
first_looking("Inspection_new")

In [ ]:
df["Inspection_new"].fillna("-", inplace=True)

In [ ]:
df["Inspection_new"].value_counts(dropna=False)

In [ ]:
df.groupby(["make_model", "body_type", "age", "Inspection_new"]).price.describe()

In [ ]:
df["Inspection_new"].replace("-", "No", inplace=True)

In [ ]:
df["Inspection_new"].value_counts(dropna=False)

In [ ]:
df["Inspection_new"].replace(["Yes", "No"], [1,0], inplace = True)

In [ ]:
df["Inspection_new"].value_counts(dropna=False)

## Upholstery_type

In [ ]:
first_looking("Upholstery_type")

In [ ]:
df["Upholstery_type"].replace(["Velour", "alcantara", "Part leather", "Full leather"], ["Cloth", "Part/Full Leather", "Part/Full Leather", "Part/Full Leather"], inplace=True)

In [ ]:
df["Upholstery_type"].value_counts(dropna=False)

In [ ]:
fill(df, "make_model", "body_type", "Upholstery_type", "ffill")

## Upholstery_color

In [ ]:
df.drop("Upholstery_color", axis=1, inplace=True)

### Nr. of Doors

In [ ]:
first_looking("Nr_of_Doors")

In [ ]:
fill(df, "make_model", "body_type", "Nr_of_Doors", "mode")

### Nr. of Seats

In [ ]:
first_looking("Nr_of_Seats")

In [ ]:
fill(df, "make_model", "body_type", "Nr_of_Seats", "mode")

### Cylinders

In [ ]:
first_looking("Cylinders")

In [ ]:
fill(df, "make_model", "body_type", "Cylinders", "mode")

In [ ]:
df.drop("Cylinders", axis = 1, inplace = True)

### Drive chain

In [ ]:
first_looking("Drive_chain")

In [ ]:
df["Drive_chain"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type", "Drive_chain"]).price.describe()

In [ ]:
cond = (df['make_model'] == "Renault Duster") & (df["body_type"] == "Off-Road")

In [ ]:
df.loc[cond,'Drive_chain'] = df.loc[cond,'Drive_chain'].replace('-','4WD')

In [ ]:
df["Drive_chain"].value_counts(dropna=False)

In [ ]:
df["Drive_chain"] = df["Drive_chain"].replace('-', np.nan)

In [ ]:
df["Drive_chain"].value_counts(dropna=False)

In [ ]:
fill(df, "make_model", "body_type", "Drive_chain", "mode")

### Emission Class

In [ ]:
first_looking("Emission_Class")

In [ ]:
df["Emission_Class"].fillna("-", inplace=True)

In [ ]:
df["Emission_Class"].value_counts(dropna=False)

In [ ]:
df.groupby(["make_model", "age", "Fuel", "Emission_Class"]).price.describe()

In [ ]:
df["Emission_Class"].replace("-", np.nan, inplace=True)

In [ ]:
df["Emission_Class"].value_counts(dropna=False)

In [ ]:
fill(df, "age", "Fuel", "Emission_Class", "ffill")

In [ ]:
df.drop("Emission_Class", axis=1, inplace=True)

### Gears

In [ ]:
first_looking("Gears")

In [ ]:
df["Gears"].fillna("-", inplace=True)

In [ ]:
df["Gears"].value_counts(dropna=False)

In [ ]:
df.groupby(["make_model", "body_type", "Gearing_Type", "Gears"]).price.describe()

In [ ]:
df["Gears"].replace([1,2,3,4,9,50,"-"], np.nan, inplace=True)

In [ ]:
df["Gears"].value_counts(dropna=False)

In [ ]:
for group1 in list(df["make_model"].unique()):
    for group2 in list(df["body_type"].unique()):
        for group3 in list(df["Gearing_Type"].unique()):
            cond1 = df["make_model"]==group1
            cond2 = (df["make_model"]==group1) & (df["body_type"]==group2)
            cond3 = (df["make_model"]==group1) & (df["body_type"]==group2) & (df["Gearing_Type"]==group3)
            mode1 = list(df[cond1]["Gears"].mode())
            mode2 = list(df[cond2]["Gears"].mode())
            mode3 = list(df[cond3]["Gears"].mode())
            if mode3 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond3]["Gears"].mode()[0])
            elif mode2 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond2]["Gears"].mode()[0])
            elif mode1 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond1]["Gears"].mode()[0])
            else:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df["Gears"].mode()[0])

In [ ]:
df["Gears"].value_counts(dropna=False)

### hp_kW

In [ ]:
first_looking("hp_kW")

In [ ]:
df["hp_kW"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type","hp_kW"]).price.describe()

In [ ]:
df["hp_kW"].replace("-", np.nan, inplace=True)

In [ ]:
fill(df, "make_model", "body_type", "hp_kW", "mode")

## Displacement_cc

In [ ]:
first_looking("Displacement_cc")

In [ ]:
df["Displacement_cc"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type","Displacement_cc"]).price.describe()

In [ ]:
df["Displacement_cc"].replace("-", np.nan, inplace=True)

In [ ]:
fill(df, "make_model", "body_type", "Displacement_cc", "mode")

## Weight_kg

In [ ]:
first_looking("Weight_kg")

In [ ]:
df["Weight_kg"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type","Weight_kg"]).price.describe()

In [ ]:
df["Weight_kg"].replace("-", np.nan, inplace=True)

In [ ]:
fill(df, "make_model", "body_type", "Weight_kg", "mode")

## CO2 Emission

In [ ]:
first_looking("CO2_Emission")

In [ ]:
df["CO2_Emission"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type","CO2_Emission"]).price.describe()

In [ ]:
df["CO2_Emission"].replace("-", np.nan, inplace=True)

In [ ]:
fill(df, "make_model", "body_type", "CO2_Emission", "median")

In [ ]:
df.drop("CO2_Emission", axis=1, inplace=True)

### Comfort_Convenience

In [ ]:
first_looking("Comfort_Convenience")

In [ ]:
fill(df, "make_model", "body_type", "Comfort_Convenience", "mode")

### Entertainment_Media

In [ ]:
first_looking("Entertainment_Media")

In [ ]:
fill(df, "make_model", "body_type", "Entertainment_Media", "mode")

### Extras

In [ ]:
first_looking("Extras")

In [ ]:
fill(df, "make_model", "body_type", "Extras", "mode")

### Safety_Security

In [ ]:
first_looking("Safety_Security")

In [ ]:
fill(df, "make_model", "body_type", "Safety_Security", "mode")

## cons_comb

In [ ]:
first_looking("cons_comb")

In [ ]:
cons_comb = (df["cons_country"] + df["cons_city"])/2

In [ ]:
df["cons_comb"] = df["cons_comb"].fillna(cons_comb)

In [ ]:
df["cons_comb"].value_counts(dropna=False)

In [ ]:
df["cons_comb"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type","cons_comb"]).price.describe()

In [ ]:
df["cons_comb"].replace([0.0, 1.0, 1.2, 1.6, 10, 11, 13.8, 32.0, 33.0, 38.0, 40.0, 43.0, 46.0, 50.0, 51.0, 54.0, 55.0, "-"], np.nan, inplace=True)

In [ ]:
df["cons_comb"].value_counts(dropna=False)

In [ ]:
fill(df, "make_model", "body_type", "cons_comb", "median")

## cons_country

In [ ]:
df.drop("cons_country", axis = 1, inplace = True)

## cons_city 

In [ ]:
df.drop("cons_city", axis = 1, inplace = True)

## End of this phase

In [ ]:
df.shape

In [ ]:
df.isnull().sum()/df.shape[0]*100

In [ ]:
df.to_csv("filled_scout.csv", index=False)